In [1]:
# Clean install
!pip uninstall -y bitsandbytes
!pip install bitsandbytes --prefer-binary --upgrade
!pip install -U transformers accelerate
!pip install -U huggingface_hub


Found existing installation: bitsandbytes 0.45.5
Uninstalling bitsandbytes-0.45.5:
  Successfully uninstalled bitsandbytes-0.45.5
  Using cached bitsandbytes-0.45.5-py3-none-manylinux_2_24_x86_64.whl.metadata (5.0 kB)
Using cached bitsandbytes-0.45.5-py3-none-manylinux_2_24_x86_64.whl (76.1 MB)


In [2]:
!pip install -q -U keras-nlp
!pip install -q -U keras>=3

In [3]:
import os

os.environ["KERAS_BACKEND"] = "jax"  # Or "torch" or "tensorflow".
# Avoid memory fragmentation on JAX backend.
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]="1.00"

In [4]:
import keras
import keras_nlp

In [5]:
#!pip install datasets
!pip install fsspec==2023.4.0



  Using cached fsspec-2023.4.0-py3-none-any.whl.metadata (6.7 kB)
Using cached fsspec-2023.4.0-py3-none-any.whl (153 kB)
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
huggingface-hub 0.31.2 requires fsspec>=2023.5.0, but you have fsspec 2023.4.0 which is incompatible.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2023.4.0 which is incompatible.


In [6]:
from datasets import load_dataset

dataset = load_dataset("databricks/databricks-dolly-15k")



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [8]:
data = []
for item in dataset['train']:
    if item["context"]:
        continue
    template = "Instruction:\n{instruction}\n\nResponse:\n{response}"
    data.append(template.format(**item))

# Use only the first 1000 examples
data = data[:1000]


In [9]:
# Show one example
print(dataset['train'][0])


{'instruction': 'When did Virgin Australia start operating?', 'context': "Virgin Australia, the trading name of Virgin Australia Airlines Pty Ltd, is an Australian-based airline. It is the largest airline by fleet size to use the Virgin brand. It commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route. It suddenly found itself as a major airline in Australia's domestic market after the collapse of Ansett Australia in September 2001. The airline has since grown to directly serve 32 cities in Australia, from hubs in Brisbane, Melbourne and Sydney.", 'response': 'Virgin Australia commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route.', 'category': 'closed_qa'}


In [10]:
!pip install transformers peft datasets accelerate

INFO: pip is looking at multiple versions of huggingface-hub to determine which version is compatible with other requirements. This could take a while.
  Using cached huggingface_hub-0.31.1-py3-none-any.whl.metadata (13 kB)
  Using cached huggingface_hub-0.31.0-py3-none-any.whl.metadata (13 kB)
  Using cached huggingface_hub-0.30.2-py3-none-any.whl.metadata (13 kB)
  Using cached huggingface_hub-0.30.1-py3-none-any.whl.metadata (13 kB)
  Using cached huggingface_hub-0.30.0-py3-none-any.whl.metadata (13 kB)
  Using cached fsspec-2025.3.2-py3-none-any.whl.metadata (11 kB)
Using cached fsspec-2025.3.2-py3-none-any.whl (194 kB)
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2023.4.0
    Uninstalling fsspec-2023.4.0:
      Successfully uninstalled fsspec-2023.4.0


In [11]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, TaskType
from datasets import Dataset
import torch


In [12]:
!pip install -U bitsandbytes


In [13]:
!pip install -q transformers accelerate bitsandbytes huggingface_hub


In [14]:
!pip install -U bitsandbytes
!pip install -U transformers accelerate




In [15]:
import torch
print(torch.cuda.is_available())


True


In [27]:

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from huggingface_hub import login

login()




In [17]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b")
model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-2b",
    quantization_config=bnb_config,
    device_map="auto"
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [21]:
print(model)


GemmaForCausalLM(
  (model): GemmaModel(
    (embed_tokens): Embedding(256000, 2048, padding_idx=0)
    (layers): ModuleList(
      (0-17): 18 x GemmaDecoderLayer(
        (self_attn): GemmaAttention(
          (q_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear4bit(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear4bit(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): GemmaMLP(
          (gate_proj): Linear4bit(in_features=2048, out_features=16384, bias=False)
          (up_proj): Linear4bit(in_features=2048, out_features=16384, bias=False)
          (down_proj): Linear4bit(in_features=16384, out_features=2048, bias=False)
          (act_fn): GELUActivation()
        )
        (input_layernorm): GemmaRMSNorm((2048,), eps=1e-06)
        (post_attention_layernorm): GemmaRMSNorm((2048,), eps=1e-06)
      )
    )
    (n

**Before Fine Tuning bold text**

In [18]:
prompt = template.format(instruction="What should I do on a trip to Europe?", response="")
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=256)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


Instruction:
What should I do on a trip to Europe?

Response:
1. You should buy a train ticket in advance.
2. You should buy a train ticket in advance.
3. You should buy a train ticket in advance.
4. You should buy a train ticket in advance.
5. You should buy a train ticket in advance.
6. You should buy a train ticket in advance.
7. You should buy a train ticket in advance.
8. You should buy a train ticket in advance.
9. You should buy a train ticket in advance.
10. You should buy a train ticket in advance.
11. You should buy a train ticket in advance.
12. You should buy a train ticket in advance.
13. You should buy a train ticket in advance.
14. You should buy a train ticket in advance.
15. You should buy a train ticket in advance.
16. You should buy a train ticket in advance.
17. You should buy a train ticket in advance.
18. You should buy a train ticket in advance.
19. You should buy a train ticket in advance.
20. You should buy a train ticket in advance.
21. You should


In [19]:
from peft import get_peft_model, LoraConfig, TaskType

# Define LoRA configuration
lora_config = LoraConfig(
    r=4,                          # LoRA rank
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],  # Typical for transformers
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM  # Because we're using a causal LM
)

# Wrap your model with LoRA
model = get_peft_model(model, lora_config)


In [20]:
model.print_trainable_parameters()


trainable params: 460,800 || all params: 2,506,633,216 || trainable%: 0.0184


In [21]:
import os
os.environ["WANDB_DISABLED"] = "true"


In [22]:
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling
from datasets import Dataset

# 1. Prepare your dataset from your list of formatted strings
train_dataset = Dataset.from_dict({"text": data})  # `data` is your list of 1000 prompts

# 2. Tokenize the dataset
def tokenize(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=512)

tokenized_dataset = train_dataset.map(tokenize, batched=True)

# 3. Data collator for language modeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# 4. Training arguments
training_args = TrainingArguments(
    output_dir="./lora-gemma-finetuned",
    per_device_train_batch_size=1,
    num_train_epochs=1,
    logging_steps=10,
    save_steps=100,
    save_total_limit=1,
    report_to=[]  # Disables WandB, TensorBoard etc.
)

# 5. Trainer setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer
)

# 6. Start training
trainer.train()


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

<ipython-input-22-37424fc1511e>:28: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
10,2.557600
20,3.066300
30,2.866000
40,2.793700
50,2.545200
60,2.446000
70,2.540300
80,2.334900
90,2.281200
100,2.486200


TrainOutput(global_step=1000, training_loss=2.1735160064697268, metrics={'train_runtime': 437.2886, 'train_samples_per_second': 2.287, 'train_steps_per_second': 2.287, 'total_flos': 6089764503552000.0, 'train_loss': 2.1735160064697268, 'epoch': 1.0})

In [26]:
prompt = template.format(instruction="What should I do on a trip to France?", response="")
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=256)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


Instruction:
What should I do on a trip to France?

Response:
France is a beautiful country with a lot to offer. You can visit the Eiffel Tower in Paris, the Louvre Museum in Paris, the Arc de Triomphe in Paris, the Palace of Versailles in Versailles, the Palace of Versailles in Versailles, the Palace of Versailles in Versailles, the Palace of Versailles in Versailles, the Palace of Versailles in Versailles, the Palace of Versailles in Versailles, the Palace of Versailles in Versailles, the Palace of Versailles in Versailles, the Palace of Versailles in Versailles, the Palace of Versailles in Versailles, the Palace of Versailles in Versailles, the Palace of Versailles in Versailles, the Palace of Versailles in Versailles, the Palace of Versailles in Versailles, the Palace of Versailles in Versailles, the Palace of Versailles in Versailles, the Palace of Versailles in Versailles, the Palace of Versailles in Versailles, the Palace of Versailles in Versailles, the Palace of Versailles in 